In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

def tune_model():
    print("Loading cleaned dataset ...")
    try:
        df = pd.read_csv("cleaned_earthquake_data.csv")
    except FileNotFoundError:
        print(" File 'cleaned_earthquake_data.csv' not found. Run data_cleaning.py first.")
        return

    print(f"Loaded {len(df)} records for tuning.")

    # Step 1 — Prepare features and target
    X = df[["Latitude", "Longitude", "Depth"]].values
    y = df["Magnitude"].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Step 2 — Build improved neural network
    model = Sequential([
        Dense(256, activation="tanh", input_shape=(3,)),
        Dropout(0.3),
        Dense(128, activation="tanh"),
        Dropout(0.2),
        Dense(64, activation="tanh"),
        Dense(1)
    ])

    # Step 3 — Compile with smaller learning rate for smoother learning
    model.compile(optimizer=Adam(learning_rate=0.0008), loss="mean_absolute_error", metrics=["mae"])

    # Step 4 — Set up callbacks
    early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint("best_earthquake_model.h5", monitor="val_loss", save_best_only=True, verbose=1)

    # Step 5 — Train model
    print(" Fine-tuning model ...")
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=150,
        batch_size=64,
        callbacks=[early_stop, checkpoint],
        verbose=1
    )

    # Load best model weights
    model.load_weights("best_earthquake_model.h5")

    # Step 6 — Evaluate performance
    print(" Evaluating tuned model ...")
    y_pred = model.predict(X_test).flatten()

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print("\n Tuned Model Performance:")
    print(f"MAE  = {mae:.4f}")
    print(f"RMSE = {rmse:.4f}")
    print(f"R²   = {r2:.4f}")

    # Step 7 — Save tuned model
    model.save("earthquake_model_tuned.h5")
    print(" Tuned model saved as 'earthquake_model_tuned.h5'")

if __name__ == "__main__":
    tune_model()

Loading cleaned dataset ...
Loaded 1000 records for tuning.
 Fine-tuning model ...
Epoch 1/150


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 1/10 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - loss: 4.6629 - mae: 4.6629
Epoch 1: val_loss improved from inf to 1.62523, saving model to best_earthquake_model.h5


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 4.0226 - mae: 4.0226 - val_loss: 1.6252 - val_mae: 1.6252
Epoch 2/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.5916 - mae: 1.5916
Epoch 2: val_loss improved from 1.62523 to 1.27096, saving model to best_earthquake_model.h5


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1844 - mae: 1.1844 - val_loss: 1.2710 - val_mae: 1.2710
Epoch 3/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.4115 - mae: 1.4115
Epoch 3: val_loss improved from 1.27096 to 0.72196, saving model to best_earthquake_model.h5


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3650 - mae: 0.3650 - val_loss: 0.3618 - val_mae: 0.3618
Epoch 11/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.3119 - mae: 0.3119
Epoch 11: val_loss improved from 0.36178 to 0.34471, saving model to best_earthquake_model.h5


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3595 - mae: 0.3595 - val_loss: 0.3447 - val_mae: 0.3447
Epoch 12/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3386 - mae: 0.3386
Epoch 12: val_loss improved from 0.34471 to 0.34353, saving model to best_earthquake_model.h5


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3562 - mae: 0.3562 - val_loss: 0.3435 - val_mae: 0.3435
Epoch 13/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.3169 - mae: 0.3169
Epoch 13: val_loss improved from 0.34353 to 0.34000, saving model to best_earthquake_model.h5


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3456 - mae: 0.3456 - val_loss: 0.3400 - val_mae: 0.3400
Epoch 14/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.4631 - mae: 0.4631
Epoch 14: val_loss did not improve from 0.34000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3699 - mae: 0.3699 - val_loss: 0.3536 - val_mae: 0.3536
Epoch 15/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.4043 - mae: 0.4043
Epoch 15: val_loss did not improve from 0.34000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3525 - mae: 0.3525 - val_loss: 0.3425 - val_mae: 0.3425
Epoch 16/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2445 - mae: 0.2445
Epoch 16: val_loss did not improve from 0.34000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3201 - mae: 0.3201 - val_loss: 0.3444 - val_mae: 0.3444
Epoch 17/150
 1/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3870 - mae: 0.3870
Epoch 17: val_loss did not improve from 0.34000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3468 -


 Tuned Model Performance:
MAE  = 0.3291
RMSE = 0.4728
R²   = -0.0708
 Tuned model saved as 'earthquake_model_tuned.h5'
